In [1]:
import pandas as pd
import csv
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.tri as tri
import math
from scipy.stats import norm
from scipy.stats import shapiro
from scipy import signal
from scipy.signal import butter
from scipy.signal import bode
import scipy.interpolate
from scipy.interpolate import RBFInterpolator
from scipy.interpolate.rbf import Rbf
import metpy
from metpy.interpolate import interpolate_to_grid
from tqdm.notebook import tqdm
import os
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
from joblib import Parallel, delayed
import time as TIME

In [2]:
##calculate minutes between utc and utc0 (time format: "YYYY-mm-ddT00:00:00Z")

def utc_to_minutes(utc, utc0):
    minutes = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").minute
    minutes0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").minute
    hours = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").hour
    hours0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").hour
    month0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").month
    month = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").month
    days = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").day
    year0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").year
    year = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").year
    
    if month0 == month:
        days -= datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").day
    if  month0!=month and year0 == year:
        if month0 == 1 or month0 ==3 or month0 == 5 or month0 == 7 or month0 == 8 or month0 == 10 or month0 == 12:
            days += 31 - datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").day
        if month0 == 2 or month0 == 4 or month0 == 6 or month0 == 9 or month0 == 11:
            days += 30 - datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").day
    
    if year0 != year:
        dmonth = month + month0%12
        i = 0
        while i <dmonth:
            if i == 0:
                if month0 == 1 or month0 ==3 or month0 == 5 or month0 == 7 or month0 == 8 or month0 == 10 or month0 == 12:
                    days += 31 - datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").day
                if month0 == 2 or month0 == 4 or month0 == 6 or month0 == 9 or month0 == 11:
                    days += 30 - datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").day
            else:
                if month0 + i == 1 or month0 + i ==3 or month0 + i== 5 or month0 + i == 7 or month0 + i == 8 or month0 + i == 10 or month0 + i == 12:
                    days += 31
                if month0 + i  == 2 or month0 + i == 4 or month0 + i == 6 or month0 + i == 9 or month0 + i == 11:
                    days += 30
            i += 1
    minutes_tot = (minutes-minutes0) + (hours-hours0)*60 + days*24*60
    
    return minutes_tot


In [3]:
##calculate utc after "minutes_" from utc0 (time format: "YYYY-mm-ddT00:00:00Z")

def minutes_to_utc(minutes_,utc0):
    seconds0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").second
    minutes0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").minute
    hours0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").hour
    month0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").month
    days0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").day
    year0 = datetime.strptime(utc0, "%Y-%m-%dT%H:%M:%SZ").year
    
    Minutes = (minutes0 + minutes_)%60
    Hours = (hours0+ (minutes0 + minutes_ )//60)%24
    a = (hours0+ (minutes0 + minutes_ )//60)//24
    days = days0 + a
    Seconds = seconds0
    month = month0
    Year = year0   
    
    if month0 == 1 or month0 ==3 or month0 == 5 or month0 == 7 or month0 == 8 or month0 == 10 or month0 == 12:
        if days > 31:            
            if month ==12:
                Year+= 1
                month = 1
                days = days%31
            else:
                month +=1
                days = days%31
    if month0 == 2 or month0 == 4 or month0 == 6 or month0 == 9 or month0 == 11:        
        if days > 30:
            month+=1
            days = days%30
            
    Y = np.int(Year)
    m = np.int(month)
    d = np.int(days)
    H = np.int(Hours)
    M = np.int(Minutes)
    S = np.int(Seconds)
    utc = datetime(Y,m,d,H,M,S)
    utc_ = utc.strftime("%Y-%m-%dT%H:%M:%SZ")

    return utc_

In [4]:
##convert time format "YYYY-mm-ddT00:00:00Z" to "YYYY-mm-dd 00:00:00"

def utc_(utc):
    seconds = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").second
    minutes = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").minute
    hours = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").hour
    month = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").month
    days = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").day
    year = datetime.strptime(utc, "%Y-%m-%dT%H:%M:%SZ").year
    Y = np.int(year)
    m = np.int(month)
    d = np.int(days)
    H = np.int(hours)
    M = np.int(minutes)
    S = np.int(seconds)
    utc = datetime(Y,m,d,H,M,S)
    utc_ = utc.strftime("%Y_%m_%d_%H_%M_%S_")
    
    return utc_

In [5]:
## in:
## floater : drifter ID
## data : data from https://www.aoml.noaa.gov/phod/gdp/index.php, csv file with format:
##        "platform_code","platform_type","time" (UTC), "latitude" (degrees north), "longitude" (degrees east), 
##        "sst" (degrees Celcius), "slp" (Hectopascal), "long360" (degrees east)
## t0 : start time (YYYY-mm-ddT00:00:00Z)
## t : end time (YYYY-mm-ddT00:00:00Z)
## lat0 , lat , long0, long: latitude and longitude range
## i : variable iterating through "data"

## out:
## floater.csv : csv file with format: "time", "lat", "long", "temp", "pressure"
##               containing all data points  of drifter "floater" between t0, t in range of lat0, lat, long0, long
## returns:
## j : number of datapoints found for drifter "floater"
## i : variable iterating through "data"


def floater_csv(floater,data, t0,t, lat0, lat, long0, long,i, path0):
    
    ## number of data points found corresponding to drifter "floater"
    j = 0
    
    ## iterate through "data" until "platform_code" corresponds to floater
    while data['platform_code'][i]!=floater and i < len(data)-1:
        i+=1
    
    header = ['time','lat','long','temp','pressure']
    ##time step between selected initial time and time of first datapoint in "data" ( in minutes)
    T0 = utc_to_minutes(t0,data['time'][1]) 
    ##time step between selected end time and time of first datapoint in "data" ( in minutes)
    T = utc_to_minutes(t,data['time'][1])

    ## iterate through "data" until t(i) is larger than T0
    while T0 > utc_to_minutes(data['time'][i],data['time'][1] ):
        i+=1
        
    ## iterate through "data", write data points into csv file "floater".csv until t(i) is larger than T
    while data['platform_code'][i]==floater and utc_to_minutes(data['time'][i],data['time'][1] ) < T:
        
        ## valid data point number "j" != 0 is added (not first)
        if j != 0 and float(data['latitude'][i])<=lat and float(data['latitude'][i])>=lat0 and float(data['longitude'][i])<=long and float(data['longitude'][i])>=long0 and utc_to_minutes(data['time'][i],data['time'][i-1])>= 60:                
            with open('%sunfiltered/%s_unfiltered.csv' % (path0,int(data['platform_code'][i])),'a',newline='') as f:            
                writer = csv.writer(f)    
                writer.writerow([data['time'][i],data['latitude'][i],data['longitude'][i],data['sst'][i],data['slp'][i]])
            j+=1
            
        ## valid data point number "j" = 0 is added (first)
        if j == 0 and float(data['latitude'][i])<=lat and float(data['latitude'][i])>=lat0 and float(data['longitude'][i])<=long and float(data['longitude'][i])>=long0:
            with open('%sunfiltered/%s_unfiltered.csv' % (path0,int(data['platform_code'][i])),'w',newline='') as f:           
                writer = csv.writer(f)                
                writer.writerow(header)                
                writer.writerow([data['time'][i],data['latitude'][i],data['longitude'][i],data['sst'][i],data['slp'][i]])       
            j+=1       
        i+=1
    
    return j,i


In [6]:
## in:
## floaters : drifter ID
## time0 : start time (YYYY-mm-ddT00:00:00Z)
## time : end time (YYYY-mm-ddT00:00:00Z)

## out:
## floaters.csv : data points with: 
##                - displacement to last smaller than 0.01 km
##                - delta t to last larger than 12h
##                - velocity to last smaller than 0.05 km/h
##                are removed (conditions for valid data point not met)
## returns:
## len(data) : number of datapoints in csv file


def filter_(floaters,time0,time, path0):
    data = pd.read_csv("%sunfiltered/%s_unfiltered.csv" % (path0,floaters))
    R = 6371
    j = 1
    k = 1
    Vel = [0]
    Dt = [0]
    noise = []
    gap = 0
    gap1 = 0
    
    ## iterate through "data" (in backwards time)
    while j <= len(data) and j + k <= len(data):
        lat0 = data['lat'][len(data)-j]
        rlat0 = math.radians(lat0)
        lat = data['lat'][len(data)-j-k]
        rlat = math.radians(lat)
        long0 = data['long'][len(data)-j]
        rlong0 = math.radians(long0)
        long = data['long'][len(data)-j-k]
        rlong = math.radians(long)
        
        ##distance between data points j, j+k (in km)
        d = 2*R*np.arcsin(np.sqrt((np.sin((rlat-rlat0)/2))**2 + np.cos(rlat0)*np.cos(rlat)*(np.sin((rlong-rlong0)/2))**2))
        
        ##time step between data points j, j+k ( in hours)
        dt = int(utc_to_minutes(data['time'][len(data)-j],data['time'][len(data)-j-k]))/60
        vel = d/dt
        
        ## if conditions for valid data point not met add data point j+k to "noise" and increment k:
        
        ## condition to obtain preprocessed drifter data:
        if dt > 12 or  round(dt) == 0 or vel > 20 or d <= 0.01:            
        ## condition to obtain raw drifter data
        #if d == 0:
        
            noise = np.append(noise,len(data)-j-k)
            k += 1
            if dt > 12:
                gap = 1
        ## if conditions met reset k to 1, increment j:
        
        ##condition to obtain preprocessed drifter data:
        if dt <= 12 and round(dt) != 0 and vel <= 20 and d > 0.01:
        ##condition to obtain raw drifter data:
        #if d != 0:
            
            Dt = np.append(Dt,dt)
            Vel = np.append(Vel,vel)
            j += 1
            if k != 1:
                j = j+k-1
            k = 1   
            
    noise = np.int_(noise)
    
    ## remove data points in "noise"
    datapoints = np.arange(0,len(data),1)
    datapoints_1 = [x for x in datapoints if x not in noise]
    data = data.drop(index = noise)
    
    ## if first/last data point more than 12 hours after/before desired start, end time 
    ## remove all data points ( drifter not in temporal range)
    if utc_to_minutes(time,data['time'][np.max(datapoints_1)])/60 > 12 or utc_to_minutes(data['time'][np.min(datapoints_1)],time0)/60 > 12:
        data = data.drop(index = datapoints_1)
        gap1 = 1
    data.to_csv("%sfiltered/%s.csv" % (path0,floaters))

    return len(data), gap, gap1

In [7]:
## from gitbhub


def filter_data(lat, long, cutoff, lat_mean, fs):
    
    for c in range(len(long)-1):
        
        # sampling frequency of drifters: 1/6hours
        #fs = f
        
        # Cut-off period
        ## filter with average latitude of drifter trajectory
        T = min(cutoff*2*np.pi/(2*7.27*10**(-5)*60*60*np.abs(np.sin(np.nanmean(list(lat))*np.pi/180))), 5*24)
        ## filter with average latitude of spatial range considered
        #T = min(cutoff*2*np.pi/(2*7.27*10**(-5)*60*60*np.abs(np.sin(lat_mean*np.pi/180))), 5*24)
        
        f = 1/T
        
        # low-pass frequency
        lowpass_frequency = f/(fs/2)
        
        # Check if there is a crossing from -180° to 180° (or viceversa) in the longitudinal position
        # and transform the longitudinal position from [-180, 180] to [0, 360]
        crossing = False
        
        for c in range(len(long)-1):
            if np.abs((long[c+1]-long[c])) > 100:
                long = np.where(long <= 0, long + 360, long)
                crossing = True
                break

        # Low-pass trajectories
        b, a = signal.butter(6, lowpass_frequency, 'low', analog = False)
        lat = signal.filtfilt(b, a, lat)
        b, a = signal.butter(6, lowpass_frequency, 'low', analog = False)
        long = signal.filtfilt(b, a, long)
        
        # If there is a crossing, then retransform coordinate back into range [-180, 180]
        if crossing:
            long = np.where(long >= 180, long - 360, long)
        
        return lat, long

In [8]:
## in : 
## floaters : drifter ID

## out :
## floaters.csv : applies filter_data() to data from "floaters.csv"

def low_pass(floaters, time0, path0, cutoff, lat_mean, f, interval_, filtering):
    data = pd.read_csv("%sfiltered/%s.csv" % (path0,floaters))
    
    ## disregards files with less than 21 data points ( required for interpolation of)
    if(len(data)<=21):
        return 1
    lat = []
    long = []
    ## read latitude longitude data into "lat", "long"
    for i in range(len(data)):
        lat = np.append(lat,data['lat'][i]) 
        long = np.append(long,data['long'][i])
    timef = [utc_to_minutes(data['time'][0],time0)]
    
    ## read time steps from initial time "time0" corresponding to "lat", "long" into "timef" (in minutes) 
    for i in range(len(data)-1):
        dt = utc_to_minutes(data['time'][i+1],data['time'][i])
        timef = np.append(timef, dt+timef[i]) 

    ##start point of interpolated data (first timestep in "time" rounded to next hour )    
    time0_ = math.ceil(utc_to_minutes(data['time'][0],time0)/60)*60
    
    ##end point of interpolated data (last timestep in "time" rounded to previous hour)
    time1_ = math.floor((timef[-1]+60)/60)*60
    
    #interpolation interval in minutes
    interval = interval_*60
    
    ##array containing timesteps at which data is interpolated with time intervals "interval"
    time = np.arange(time0_,time1_,interval) 
    time11 = np.arange(time0_,time1_,60)
    
    ##interpolate latitude, longitude at time steps in "time" into lat_interp, long_interp
    lat_interp1 = scipy.interpolate.interp1d(timef[:], lat[:])
    lat_interp = lat_interp1(time[:])   
    long_interp1 = scipy.interpolate.interp1d(timef[:], long[:])
    long_interp = long_interp1(time[:])
    
    ## filter interpolated data
    if filtering:
        lat_filt, long_filt = filter_data(lat_interp,long_interp, cutoff, lat_mean, f)
    else:
        lat_filt, long_filt = lat_interp, long_interp
    #
    
    ## read interpolated, filtered data back into csv file
    for i in range(len(time)):
        data.loc[i,('lat')] = lat_filt[i]
        data.loc[i,('long')] = long_filt[i]
        data.loc[i,('time')] = minutes_to_utc(time[i],time0)
    i = 0
    for i in range(len(time), len(time11)):
        data = data.iloc[:i]
    data.to_csv("%sfiltered/%s.csv" % (path0,floaters),columns = ['time', 'lat', 'long', 'temp', 'pressure'])


In [9]:
## main:
## data : all drifter data from x.csv
## time0 , time: temporal range of interest
## lat0, lat, long0, long : spatial range considered
## floaters : array containing all floater IDs found in "data"
## - extracts all drifter IDs from "data"
## - extracts drifter data from "data" with floaters_csv() until desired number of floaters found in temporal, 
##   spatial range of interest
## - if more than 21 datapoints found: (minimal number required for interpolation)
## - applies filter_() to csv files of drifters
## - applies low_pass() to csv files of drifters
##############################################################################################################

##parameters:
##spatial range of region of interest in Degrees
long0 = -50
long = 10
lat0 = 30
lat = 90

##temporal range of region of interest
#time0 = '2022-09-10T00:00:00Z'
time0 = '2022-08-23T00:00:00Z' ##starttime
#time = '2022-08-30T00:00:00Z' ##end time 7days
#time = '2022-09-06T00:00:00Z' ##end time 14days
time = '2022-09-13T00:00:00Z' ##end time 21days
#time = '2022-09-20T00:00:00Z' ##end time 28days


##########################
##filtering parameters:

##factor of cutoff frequency:
#cutoff = 0.5
#cutoff = 1.0
cutoff = 1.5
#cutoff = 1.25
#cutoff = 2.0
#cutoff = 2.5
#cutoff = "inf"

##sampling period (h)
interval_ = 6
##if low pass filter applied: "filtering" = "True" 
filtering = True

################################################################################################################
##Data Locations:
##-path
##--path_folders
##---path_data
##----path0
##-----path_filtered
##-----path_unfiltered
##----raw_download

##Path in PC
path = "C:/Mario/TBarrier/TBarrier/Untitled Folder/"

##Folder "TRA_data" in "path"
path_folders = "%sTRA_data/" % path

##Folder with spatial range in "TRA_data"
path_data = "%slong_%s_%s_lat_%s_%s_/" % ( path_folders ,long0, long, lat0, lat)

##Folder with Timeframe, filtering parameters in "path_data":

##time interval between "time0" and "time" in days
DD = np.int_(utc_to_minutes(time,time0)/60/24)

if filtering == True:
    path0 = "%stime0_%s_%s_days/filter_%s_cutoff_%s/sampling period_%s/" % (path_data,utc_(time0), DD,filtering, cutoff, interval_ )
else:
    path0 = "%stime0_%s_%s_days/filter_%s/sampling period_%s/" % (path_data,utc_(time0), DD,filtering, interval_)


##Folder with filtered, unfiltered data
path_filtered = "%sfiltered/" % path0
path_unfiltered = "%sunfiltered/" %path0

##Path of folders where filtered, unfiltered drifter csv files are saved in folder "x"
Path('%s' % path_filtered).mkdir(parents=True, exist_ok=True)
Path('%s' % path_unfiltered).mkdir(parents=True, exist_ok=True)
#######################################
##Download, Read data:

##records time program takes to run
start_time = TIME.time()

##dowload drifter data from GDP:
#website_path = "http://osmc.noaa.gov/erddap/tabledap/OSMC_30day.csv?platform_code%2Cplatform_type%2Ctime%2Clatitude%2Clongitude%2Csst%2Cslp%2Clon360&platform_type=%22DRIFTING%20BUOYS%20(GENERIC)%22&orderBy(%22platform_code%2Ctime%22)"
#data = pd.read_csv(website_path)

##read data from downloaded file:
file_path = "OSMC_30day_0133_b0b8_ef07.csv"
data = pd.read_csv(file_path)

data.to_csv("%sraw_download.csv" % path_folders , columns = ['platform_code', 'platform_type', 'time', 'latitude', 'longitude', 'sst', 'slp', 'lon360' ])
#######################################
##Extract drifters in range:

## extract all drifter IDs in "data" into "floaters"
floaters = [int(data['platform_code'][1])]
for i in tqdm(range(2, len(data['platform_code'])-1), desc = "Finding Drifter IDs"):
    if data['platform_code'][i]!= data['platform_code'][i+1]:
        floaters = np.append(floaters,int(data['platform_code'][i]))

## "index" iterates through "data" to extract drifter data (reduces computation time by starting extraction of next drifter at end of
## previously extracted drifter)
index = 0

##records number of extracted drifters: 
k = 0
##records number of drifters matching spatial, temporal range at any timestep
a = 0
b = 0
c = 0
gap_ = 0
gap_1 = 0
## end latitude, longitude, TRA value of extracted drifters in "Floaters":
lat_c = []
long_c = []
TRA = []
Floaters = []

##average latitude of Region of interest
lat_mean = (lat0+lat)/2

##sampling frequency low pass filter (1/6h)
fs = 6/interval_

##iterate through "data" extracting drifters:
#while i in range(len(floaters)) and k < floaters_nr:
for i in tqdm(range(len(floaters)), desc = "Extracting Drifters"):
    
    ## read data corresponding to drifter "floater[i]" between "time0" and "time" in range of "lat0", "lat", "long0", "long"
    ##into csv file "floaters[i]_unfiltered".csv:
    j, index = floater_csv(floaters[i],data, time0, time, lat0, lat, long0, long, index, path0)  
    if j >= 1:
        a+= 1
    ## if more than 21 data points in csv file "floaters[i]"_unfiltered.csv data is filtered:
    if j >21: 
        b +=1
        ##if more than 21 data points remaining after data points not corresponding to drifter trajectory are removed
        ## the remaining data points are interpolated, low pass filtered and read into "floaters[i]".csv:
        len_data, gap , gap1 = filter_(floaters[i],time0,time, path0)
        gap_ += gap
        gap_1 += gap1
        if len_data > 21:
            c+= 1
            Floaters = np.append(Floaters,floaters[i])         
            low_pass(floaters[i], time0, path0, cutoff, lat_mean , fs, interval_, filtering)            
            k+=1
            
        ## if less than 21 data points remain all files corresponding to drifter "floaters[i]" are deleted
        else:
            os.remove("%s%s.csv" % (path_filtered,floaters[i]))
            os.remove("%s%s_unfiltered.csv" % (path_unfiltered,floaters[i]))
        
    elif j <=21 and j > 0:
        os.remove("%s%s_unfiltered.csv" % (path_unfiltered,floaters[i]))
    i+=1
print(len(floaters),"drifters in download file")
print(a, "drifters match spatial, temporal range")
print(a-b, "drifters with less than 21 data points")
print(gap_,"drifter with gap larger than 12 hours")
print(gap_1,"drifters starting or stopping during interval")
print(b-c, "drifters with less than 21 data points after applying conditions")
print(k,"valid drifters found")
print("--- %s seconds ---" % (TIME.time() - start_time))


C:\Mario\anaconda3\envs\TBarrier\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (3,4,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Finding Drifter IDs:   0%|          | 0/2884387 [00:00<?, ?it/s]

Extracting Drifters:   0%|          | 0/1812 [00:00<?, ?it/s]

1812 drifters in download file
316 drifters match spatial, temporal range
19 drifters with less than 21 data points
7 drifter with gap larger than 12 hours
35 drifters starting or stopping during interval
35 drifters with less than 21 data points after applying conditions
262 valid drifters found
--- 261.0102560520172 seconds ---


In [10]:

"""
from tqdm.notebook import tqdm

for i in tqdm(range(199990)):
    pass
"""

'\nfrom tqdm.notebook import tqdm\n\nfor i in tqdm(range(199990)):\n    pass\n'

## 

### 